In [576]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2023-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

In [577]:
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
mind=temp[temp['Fav_Odds']<1.3].copy()
mind=mind[mind['Fav_Odds']>1]
mind=mind[mind['Date']>='2024-01-01']

favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()
mind=mind[(mind['Fav']).isin(fav_list)]
mind=mind[mind['dog_rank']>50]
len(mind[mind['Fav']==mind['Winner']])/len(mind),len(mind)
mind[['Fav','Dog','Fav_Odds','Winner','Fav_Win','dog_rank']].to_clipboard(index=False)

In [578]:
temp
date_filter=temp[temp['Date']>='2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.5].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
oddsfilter=oddsfilter[oddsfilter['dog_percent']>0]
#oddsfilter=oddsfilter[oddsfilter['fav_percent']<.8]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Fav']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter

Total Profit: 3375.0 97 0.6185567010309279


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Profit
5347,2025-03-16,2.06,Irina Bara,1.72,2.06,Ipek Oz,Ipek Oz,10,Irina Bara,12,...,0.0,163.0,104.0,False,True,1,2,1,1,106.0
5377,2025-03-18,2.11,Lourdes Carle Maria,1.69,2.11,Sara Bejlek,Sara Bejlek,14,Lourdes Carle Maria,21,...,0.5,114.0,71.0,False,True,1,3,1,5,111.0
5396,2025-03-19,1.58,Tamara Zidansek,1.58,2.29,Tamara Zidansek,Tamara Zidansek,10,Noma Noha Akugue,11,...,0.2,22.0,142.0,True,False,0,1,0,5,-100.0
5411,2025-03-19,1.91,Clara Burel,1.84,1.91,Nuria Parrizas Diaz,Nuria Parrizas Diaz,12,Clara Burel,11,...,0.6,45.0,42.0,False,True,1,4,1,3,91.0
5422,2025-03-20,1.54,Clara Burel,1.54,2.41,Clara Burel,Clara Burel,12,Darja Semenistaja,21,...,0.1,42.0,106.0,True,False,0,2,0,6,-100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984,2025-05-08,2.47,Elisabetta Cocciaretto,1.54,2.47,Elina Avanesyan,Elina Avanesyan,17,Elisabetta Cocciaretto,16,...,0.4,36.0,29.0,False,True,1,4,1,3,147.0
6988,2025-05-08,1.76,Anastasia Potapova,1.76,2.06,Anastasia Potapova,Anastasia Potapova,17,Dayana Yastremska,11,...,0.2,21.0,21.0,True,False,0,3,0,1,-100.0
6990,2025-05-08,1.85,Peyton Stearns,1.85,1.94,Peyton Stearns,Peyton Stearns,14,Anna Kalinskaya,10,...,0.4,42.0,11.0,True,False,0,5,0,2,-100.0
6994,2025-05-08,2.27,Jaqueline Cristian,1.63,2.27,Yulia Putintseva,Yulia Putintseva,12,Jaqueline Cristian,15,...,0.5,20.0,57.0,False,True,1,2,1,3,127.0


In [608]:
temp
date_filter=temp[temp['Date']>'2024-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
#oddsfilter=date_filter[date_filter['Dog_Odds']<2]
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']<200]
oddsfilter=oddsfilter[oddsfilter['dog_percent']>0.5]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter

Total Profit: 8764.0 324 0.6234567901234568


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Profit
9,2024-02-11,2.27,Facundo Bagnis,1.63,2.27,Federico Coria,Facundo Bagnis,10,Facundo Bagnis,15,...,0.8,49.0,55.0,False,True,1,1,1,1,127.0
24,2024-02-14,2.0,Alejandro Tabilo,1.80,2.00,Yannick Hanfmann,Alejandro Tabilo,11,Alejandro Tabilo,11,...,1.0,45.0,49.0,False,True,1,1,1,1,100.0
28,2024-02-15,2.29,Facundo Diaz Acosta,1.62,2.29,Francisco Cerundolo,Facundo Diaz Acosta,12,Facundo Diaz Acosta,11,...,0.2,19.0,87.0,False,True,1,1,1,2,129.0
35,2024-02-17,2.02,Facundo Diaz Acosta,1.80,2.02,Dusan Lajovic,Facundo Diaz Acosta,9,Facundo Diaz Acosta,12,...,0.3,23.0,87.0,False,True,1,1,1,3,102.0
42,2024-02-18,1.97,Alberto Olivieri Genaro,1.80,1.97,Lucas Reis Da Silva Joao,Alberto Olivieri Genaro,10,Alberto Olivieri Genaro,10,...,0.5,259.0,131.0,False,True,1,1,1,1,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6897,2025-05-06,2.08,Nicolas Moreno De Alboran,1.72,2.08,Giulio Zeppieri,Nicolas Moreno De Alboran,14,Nicolas Moreno De Alboran,11,...,0.3,110.0,107.0,False,True,1,9,1,6,108.0
6904,2025-05-06,1.77,Giulio Zeppieri,1.77,1.99,Giulio Zeppieri,Pavel Kotov,15,Pavel Kotov,10,...,0.4,110.0,50.0,True,False,0,9,0,3,-100.0
6917,2025-05-07,1.65,Vilius Gaubas,1.65,2.18,Vilius Gaubas,Hugo Dellien,16,Hugo Dellien,38,...,0.3,144.0,64.0,True,False,0,11,0,5,-100.0
6930,2025-05-07,1.97,Camilo Ugo Carabelli,1.83,1.97,Pablo Carreno Busta,Camilo Ugo Carabelli,10,Camilo Ugo Carabelli,37,...,0.5,10.0,59.0,False,True,1,2,1,10,97.0


In [580]:
date_filter=pd.read_pickle(r'.\Clay_Today')
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()

oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>0]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Fav']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [581]:
date_filter=pd.read_pickle(r'.\Clay_Today')
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>2]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>2]
oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count
25,20:00,1.61,2.31,Frances Tiafoe,Sebastian Ofner,50%,4,Sebastian Ofner,46%,13,...,0.5,0.4,0.81,0.19,0.69,0.16,0.4,0.6,10.0,7.0


In [582]:
lowodds=pd.read_pickle(r'.\Clay_Today')
lowodds=lowodds[lowodds['Fav_Odds']<1.3].copy()
lowodds=lowodds[lowodds['Fav_Odds']>1]
favs=pd.read_csv('Fav_Clay.csv')
fav_list=favs['Name'].tolist()
lowodds=lowodds[(lowodds['Fav']).isin(fav_list)]
lowodds=lowodds[lowodds['dog_rank']>50]
lowodds

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count
0,00:10,1.04,11.94,Mirra Andreeva,Mirra Andreeva,87%,15,Emiliana Arango,0%,2,...,0.7,0.4,0.56,0.21,0.00,0.00,0.6,0.4,7.0,5.0
8,02:00,1.12,6.27,Emma Navarro,Emma Navarro,61%,18,Kamilla Rakhimova,29%,7,...,0.4,0.4,0.51,0.29,0.60,0.33,0.4,0.6,10.0,10.0
15,04:15,1.14,5.68,Elena Rybakina,Elena Rybakina,87%,15,Eva Lys,25%,4,...,0.8,0.4,0.78,0.30,0.50,0.39,0.6,0.4,2.0,4.0
24,20:00,1.25,3.94,Tommy Paul,Tommy Paul,67%,15,Roberto Bautista Agut,54%,13,...,0.6,0.5,0.80,0.24,0.82,0.27,0.6,0.4,3.0,4.0
36,23:00,1.18,4.81,Elina Svitolina,Elina Svitolina,90%,21,Hailey Baptiste,20%,5,...,0.7,0.3,0.64,0.37,0.64,0.29,0.8,0.8,4.0,2.0


In [ ]:
# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)  
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

today=pd.read_pickle(r'.\Clay_Today')
today=today[today['Fav_Odds']>1.6]
today=today[today['Dog_Win_Count']>2]
today=today[today['Fav_Loss_Count']>2]
today=today[today['Sex']=='Mens']
today['Date']=date_string
today.to_sql('Preditions_Clay',con=devengine,if_exists='append')
today=pd.read_pickle(r'.\Clay_Today')
today=today[today['Fav_Odds']>1.4]
today=today[today['Dog_Win_Count']>0]
today=today[today['Fav_Loss_Count']>0]
today=today[today['Sex']=='Womens']
today['Date']=date_string
today.to_sql('Preditions_Clay',con=devengine,if_exists='append')

In [614]:
predictions=pd.read_sql_table('Preditions_Clay',con=devengine)
results=temp

# Ensure date types
predictions['Date'] = pd.to_datetime(predictions['Date'])
results['Date'] = pd.to_datetime(results['Date'])

# Step 1: Initial merge on the exact date
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],
    on=['Date', 'Fav', 'Dog'], 
    how='left'
)

# Step 2: Get unmatched rows only
unmatched = merged_df[merged_df['Winner'].isna()].copy()

if not unmatched.empty:
    # Try matching with results where match happened one day after the prediction date
    results_shifted = results.copy()
    results_shifted['Date'] = results_shifted['Date'] - pd.Timedelta(days=1)

    fallback = pd.merge(
        unmatched.drop(columns=['Winner']),  # remove old (NaN) winner
        results_shifted[['Date', 'Fav', 'Dog', 'Winner']],
        on=['Date', 'Fav', 'Dog'],
        how='left'
    )

    # Only update original merged_df where Winner was missing AND fallback found a match
    merged_df.loc[merged_df['Winner'].isna(), 'Winner'] = fallback['Winner'].values


# merged_df now has best effort match: same day, or next day fallback
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>1.6].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['dog_percent']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Dog_Win_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Loss_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Sex']=='Womens']
merged_df_filtered=merged_df_filtered[merged_df_filtered['Elo_Fav']==merged_df_filtered['Fav']]

merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered.tail(5)

Total Profit: 556.0 32 0.5625


,Date,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,...,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count,Winner,Profit
292,2025-05-06,4:15,1.72,2.07,Lourdes Carle Maria,Lourdes Carle Maria,55%,22,Maya Joint,67%,...,NaN,NaN,NaN,NaN,0.6,0.6,7,4,Maya Joint,107.0
302,2025-05-07,20:00,1.63,2.26,Camila Osorio,Camila Osorio,53%,19,Victoria Azarenka,58%,...,0.57,0.37,0.67,0.40,0.8,0.4,1,1,Victoria Azarenka,126.0
320,2025-05-08,2:50,1.72,2.11,Anastasia Pavlyuchenkova,Anastasia Pavlyuchenkova,38%,13,Katie Boulter,50%,...,0.65,0.43,0.69,0.30,0.2,0.2,7,1,Anastasia Pavlyuchenkova,-100.0
321,2025-05-08,6:15,1.76,2.06,Anastasia Potapova,Anastasia Potapova,59%,17,Dayana Yastremska,9%,...,0.63,0.42,0.65,0.28,0.8,0.8,4,1,Anastasia Potapova,-100.0
347,2025-05-09,4:40,1.82,1.96,Marie Bouzkova,Marie Bouzkova,44%,9,Beatriz Haddad Maia,60%,...,0.73,0.39,0.72,0.31,0.6,0.4,3,5,Marie Bouzkova,-100.0


In [611]:
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>1.6].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['dog_percent']>0.5]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Dog_Win_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Loss_Count']>0]
merged_df_filtered=merged_df_filtered[merged_df_filtered['Sex']=='Mens']
merged_df_filtered=merged_df_filtered[merged_df_filtered['Elo_Fav']==merged_df_filtered['Dog']]

merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered.tail(5)

Total Profit: 301.0 31 0.5483870967741935


,Date,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,...,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count,Winner,Profit
307,2025-05-07,20:00,1.67,2.19,Jordan Thompson,Giovanni Mpetshi Perricard,33%,6,Giovanni Mpetshi Perricard,62%,...,0.77,0.16,0.00,0.00,0.2,0.2,4,5,Jordan Thompson,-100.0
314,2025-05-07,19:30,1.72,2.00,Nicolai Budkov Kjaer,Calvin Hemery,40%,5,Calvin Hemery,57%,...,NaN,NaN,0.00,0.00,0.2,0.6,17,19,Nicolai Budkov Kjaer,-100.0
319,2025-05-07,0:20,1.65,2.18,Vilius Gaubas,Hugo Dellien,50%,16,Hugo Dellien,58%,...,NaN,NaN,0.72,0.20,0.6,0.4,12,4,Vilius Gaubas,-100.0
336,2025-05-08,6:05,1.87,1.93,Vilius Gaubas,Damir Dzumhur,12%,8,Damir Dzumhur,69%,...,NaN,NaN,0.67,0.24,0.6,0.4,12,13,Vilius Gaubas,-100.0
341,2025-05-09,1:15,1.84,1.86,Francesco Maestrelli,Radu Albot,43%,14,Radu Albot,56%,...,0.67,0.08,0.78,0.21,0.6,0.2,24,6,Francesco Maestrelli,-100.0
